In [11]:
## Packages
using LinearAlgebra
using Colors
using Printf


## Basic functions
function DeleteDuplicates(v::Array{T,1})::Array{T,1} where T <: Any
    w=Array{eltype(v),1}()
    if (length(v)>0)
        for e ∈ v
            if e ∉ w
                push!(w,e)
            end
        end
    end
    return w
end

function OrthogonalVector(v::Array{T,1}) where T <: Real
    if(norm(v)==0)
        error("zero vector")
    else
        v₁=normalize(v)
        i=argmin(abs.(v))
        v₂=zeros(length(v))
        v₂[i]=1.0
        v₂=normalize(v₂-dot(v₁,v₂)*v₁)
        return v₂
    end
end

function Circumcenter(point₁,point₂,point₃)
    l₁²=(norm(point₂-point₃))^2
    l₂²=(norm(point₃-point₁))^2
    l₃²=(norm(point₁-point₂))^2
    c₁=l₁²*(l₂²+l₃²-l₁²)
    c₂=l₂²*(l₃²+l₁²-l₂²)
    c₃=l₃²*(l₁²+l₂²-l₃²)
    center=(c₁*point₁+c₂*point₂+c₃*point₃)/(c₁+c₂+c₃)
    return center
end

function NormalVector(v₁::Array{Float64,1},v₂::Array{Float64,1},v₃::Array{Float64,1})
    return normalize(cross(v₁,v₂)+cross(v₂,v₃)+cross(v₃,v₁))
end


## Types
abstract type JuliRay end
abstract type Object <: JuliRay end
abstract type PrimitiveObject <: Object end
abstract type csgObject <: Object end
abstract type ColoredObject <: Object end
abstract type TransformedObject <: Object end

struct FT <: JuliRay
    filter::Float64
    transmit::Float64
    FT(filter,transmit) =
    if(filter<0 || transmit<0 || filter+transmit>1)
        error("だめです")
    else
        new(filter,transmit)
    end
end





## Primitive shapes
struct Empty <: PrimitiveObject end
struct Sphere <: PrimitiveObject
    center :: Array{Float64,1}
    radius :: Float64
    Sphere(center,radius) = (radius ≠ 0.0) ? new(center,abs(radius)) : Empty
end
struct Cylinder <: PrimitiveObject
    end1 :: Array{Float64,1}
    end2 :: Array{Float64,1}
    radius :: Float64
    Cylinder(end1,end2,radius) = (norm(end2-end1)*radius ≠ 0) ? new(end1,end2,abs(radius)) : Empty
end
struct Cone <: PrimitiveObject
    end1 :: Array{Float64,1}
    end2 :: Array{Float64,1}
    radius :: Float64
    Cone(end1,end2,radius) = (norm(end2-end1)*radius ≠ 0) ? new(end1,end2,abs(radius)) : Empty
end
struct Box <: PrimitiveObject
    vertex1 :: Array{Float64,1}
    vertex2 :: Array{Float64,1}
    Box(vertex1,vertex2) = (norm(vertex2-vertex1) ≠ 0) ? new(vertex1,vertex2) : Empty
end
struct Disc <: PrimitiveObject
    center :: Array{Float64,1}
    normal :: Array{Float64,1}
    radius :: Float64
    Disc(center,normal,radius) =
    if(norm(normal) == 0)
        error()
    elseif(radius == 0)
        Empty
    else
        new(center,normal,abs(radius))
    end
end
struct Torus <: PrimitiveObject
    radius1 :: Float64
    radius2 :: Float64
    Torus(radius1,radius2) =
    if(radius2 == 0)
        Empty
    elseif(radius1 == 0)
        Sphere([0,0,0],radius2)
    else
        new(radius1,radius2)
    end
end
struct Polygon <: PrimitiveObject
    vertices :: Array{Array{Float64,1},1}
    Polygon(vertices) =
    if(rank(hcat(vertices...)-repeat(+(vertices...)/length(vertices),1,length(vertices))) ≠ 2)
        Empty
    else
        new(vertices)
    end
end

struct csgUnion <: csgObject
    objects :: Array{Object,1}
    csgUnion(objects) =
    if(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 0)
        Empty
    elseif(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 1)
        objects[1]
    else
        new(DeleteDuplicates(deleteat!(objects, objects.== Empty)))
    end
end
struct csgIntersection <: csgObject
    objects :: Array{Object,1}
    csgIntersection(objects) =
    if(Empty ∈ objects)
        Empty
    elseif(length(DeleteDuplicates(objects)) == 0)
        Empty
    elseif(length(DeleteDuplicates(objects)) == 1)
        objects[1]
    else
        new(DeleteDuplicates(objects))
    end
end
struct csgMerge <: csgObject
    objects :: Array{Object,1}
    csgMerge(objects) =
    if(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 0)
        Empty
    elseif(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 1)
        objects[1]
    else
        new(DeleteDuplicates(deleteat!(objects, objects.== Empty)))
    end
end
struct csgDifference <: csgObject
    objects :: Array{Object,1}
    csgDifference(objects) =
    if(length(objects) ≠ 2)
        error("だめです")
    elseif(objects[1] == Empty)
        Empty
    elseif(objects[2] == Empty)
        objects[1]
    else
        new(objects)
    end
end

# Transform object to object
struct rgbColor <: ColoredObject
    object :: Object
    color :: Color
    rgbColor(object,color) = (object ≠ Empty) ? new(object,color) : Empty
end
struct rgbftColor <:ColoredObject
    object :: Object
    color :: Color
    transparence :: FT
    rgbftColor(object,color,ft) = (object ≠ Empty) ? new(object,color,ft) : Empty
end


struct AffineTransform <:TransformedObject
    object :: Object
    A :: Array{Float64,2}
    b :: Array{Float64,1}
    AffineTransform(object,A,b) = (object ≠ Empty && det(A) ≠ 0) ? new(object,A,b) : Empty
end





## 文字列変換
function vector2pov(x::Array{T,1}) where T <: Real
    "<"*repr(x)[2:end-1]*">"
end

function object2pov(sphere :: Sphere)
    return "sphere{"*vector2pov(sphere.center)*","*repr(sphere.radius)*"}"
end
function object2pov(cylinder :: Cylinder)
    return "cylinder{"*vector2pov(cylinder.end1)*","*vector2pov(cylinder.end2)*","*repr(cylinder.radius)*"}"
end
function object2pov(cone :: Cone)
    return "cone{"*vector2pov(cone.end1)*","*repr(cone.radius)*","*vector2pov(cone.end2)*",0}"
end
function object2pov(box :: Box)
    return "box{"*vector2pov(box.vertex1)*","*vector2pov(box.vertex2)*"}"
end
function object2pov(disc :: Disc)
    return "disc{"*vector2pov(disc.center)*","*vector2pov(disc.normal)*","*repr(disc.radius)*"}"
end
function object2pov(torus :: Torus)
    return "torus{"*repr(torus.radius1)*","*repr(torus.radius2)*"}"
end
function object2pov(polygon :: Polygon)
    n=length(polygon.vertices)
    return "polygon{"*repr(n)* (*(reshape([(repeat([","],n),vector2pov.(polygon.vertices))[i][j] for i in 1:2, j in 1:n],2n)...))*"}"
end
# csg
function object2pov(csg :: csgUnion)
    return "union{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgIntersection)
    return "intersection{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgMerge)
    return "merge{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgDifference)
    return "difference{"* *(object2pov.(csg.objects)...)*"}"
end
# Transform
function object2pov(rgbcolor :: rgbColor)
    r=string(Float64(rgbcolor.color.r))
    g=string(Float64(rgbcolor.color.g))
    b=string(Float64(rgbcolor.color.b))
    return "object{"*object2pov(rgbcolor.object)*" pigment{rgb<"*r*","*g*","*b*">}}"
end
function object2pov(rgbftcolor :: rgbftColor)
    r=string(Float64(rgbftcolor.color.r))
    g=string(Float64(rgbftcolor.color.g))
    b=string(Float64(rgbftcolor.color.b))
    f=string(Float64(rgbftcolor.transparence.filter))
    t=string(Float64(rgbftcolor.transparence.transmit))
    return "object{"*object2pov(rgbftcolor.object)*" pigment{rgbft<"*r*","*g*","*b*","*f*","*t*">}}"
end
function object2pov(affinetransform :: AffineTransform)
    return "object{"*object2pov(affinetransform.object)*" matrix"*vector2pov(vcat(reshape(affinetransform.A,9),affinetransform.b))*"}"
end

## Verargs objects
function Polygon(vertices...)
    return Polygon([vertices...])
end
function csgUnion(objects::Object...)
    return csgUnion([objects...])
end
function csgIntersection(objects::Object...)
    return csgIntersection([objects...])
end
function csgMerge(objects::Object...)
    return csgMerge([objects...])
end
function csgDifference(object1,object2)
    return csgDifference([object1,object2])
end


## Compound objects
function Arrow(end1, end2, radius)
    n=normalize(end2-end1)
    end3=end2-6*radius*n
    return csgUnion(Cylinder(end1,end3,radius),Cone(end3,end2,2*radius))
end
function Torus(center::Array{Float64,1}, normal::Array{Float64,1}, radius1::Float64, radius2::Float64)
    e₁=normalize(normal)
    e₂=OrthogonalVector(e₁)
    e₃=cross(e₁,e₂)
    A=hcat(e₃,e₁,e₂)
    return AffineTransform(Torus(radius1,radius2),A,center)
end
function Torus(point1::Array{Float64,1}, point2::Array{Float64,1}, point3::Array{Float64,1}, radius::Float64)
    center=Circumcenter(point1,point2,point3)
    normal=NormalVector(point1,point2,point3)
    R=norm(point1-center)
    return Torus(center,normal,R,radius)
end
function Disc(point1::Array{Float64,1}, point2::Array{Float64,1}, point3::Array{Float64,1})
    center=Circumcenter(point1,point2,point3)
    normal=NormalVector(point1,point2,point3)
    R=norm(point1-center)
    return Disc(center,normal,R)
end
function Blocks³(point1::Array{Float64,1}, point2::Array{Float64,1}, point3::Array{Float64,1}, thickness::Float64)
    center=Circumcenter(point1,point2,point3)
    e₃=NormalVector(point1,point2,point3)
    R=norm(point1-center)
    e₁=normalize(point1-center)
    e₂=cross(e₃,e₁)
    box=Box([R+thickness,R+thickness,thickness],-[R+thickness,0,thickness])
    box1=AffineTransform(box,hcat(e₁,e₂,e₃),center)
    e₁=-normalize(point3-center)
    e₂=cross(e₃,e₁)
    box2=AffineTransform(box,hcat(e₁,e₂,e₃),center)
    # println(det(hcat(point1-center,point3-center,e₃)))
    if(det(hcat(point1-center,point3-center,e₃))>0)
        return csgIntersection(box1,box2)
    else
        return csgMerge(box1,box2)
    end
end
function Arc(point1::Array{Float64,1}, point2::Array{Float64,1}, point3::Array{Float64,1}, radius::Float64)
    return csgIntersection(Torus(point1,point2,point3,radius),Blocks³(point1,point2,point3,2radius))
end


function Transparent(rgbcolor::rgbColor,ft::FT)
    return rgbftColor(rgbcolor.object,rgbcolor.color,ft)
end


# Rendering
function render(obj::T;name="new", index::Int=0, width::Int=500, height::Int=500) where T <: Object
    if(0<index<1000000)
        Index="_"*(@sprintf "%06d" index)
    elseif(index==0)
        Index=""
    else
        error("index must be non-negative and less than 1000000")
    end
    if(endswith(name,".pov"))
        Name=name[1:end-4]*Index*".pov"
    else
        Name=name*Index*".pov"
    end
    
    str="#version 3.7;\nglobal_settings{assumed_gamma 1.0}\n\n#include \"Hy_constants.inc\"\n#include \"Hy_functions.inc\"\n#include \"Hy_colors.inc\"\n\n#declare Lng=30;\n#declare Lat=30;\n#declare Pers=0.1;\n#declare Zoom=0.9;\n#declare LookAt=<0,0,0>;\n#include \"Hy_camera.inc\"\n\n"
    str=str*object2pov(obj)
    open(Name, "w") do io
      println(io, str)
    end
    run(`povray $Name`)
end

render (generic function with 1 method)

In [12]:
# q=Blocks³([0,1.,0],[0,1.,1.],[0,0,1.0],0.03)
ttt=FT(0.5,0.5)
q=Transparent(rgbColor(Arc([0,1.,0],[0,0,1.0],[0,1.,1.],0.03),RGB(1,0,0)),ttt)
render(q,index=12345)

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray new_012345.pov`, ProcessExited(0))

In [33]:
n=10
for iii ∈ 0:(n-1)
    t=iii/n
    θ=π*(1+cos(2π*t))/4
    A=[1 0 0;0 cos(θ) -sin(θ);0 sin(θ) cos(θ)]
    A2=[cos(θ) -sin(θ) 0;sin(θ) cos(θ) 0;0 0 1]
    o=Sphere([0,0,0],0.05)
    x=rgbColor(Arrow([0,0,0],[1,0,0],0.02+0.01*cos(2π*t)),[1,0,0])
    y=rgbColor(Arrow([0,0,0],[0,1,0],0.02+0.01*cos(2π*(t+1/3))),[0,1,0])
    z=rgbColor(Arrow([0,0,0],[0,0,1],0.02+0.01*cos(2π*(t+2/3))),[0,0,1])
    a=Cylinder([0,0,0],[0,0,1],-0.1)
    b=rgbColor(Cone([-1,0,0],[-0.8,0.5,1],-0.1),[1,1,1/2-sin(2π*t)/2])
    c=Box([1,1,1]*0.9,[0,0.7+0.15*sin(2π*t),0.4])
    d=AffineTransform(rgbColor(a,[1,0.2,0.3]),[1 0 0;0 1 0;0 0 1],[0,-0.7,-0.3+0.3cos(2π*t)])
    e=rgbColor(c,[0.2,0.3,0.5])
    f=rgbColor(Torus(0.6,0.15),[0.9,0.1,0.15])
    g=AffineTransform(f,A,[0,0,0])
    h=AffineTransform(rgbColor(Torus(0.6,0.1),[0.2,0.8,0.1]),A2,[0,0,0])
    i=[rgbColor(Disc([1,ii+0.1t,-ii-0.1t],[0.2,-0.4,1],0.1),[0,.8,.8]) for ii ∈ collect(range(-1,1,step=0.1))]
    j=[rgbColor(Torus([1,ii+0.1t,-ii-0.1t],[0.2,-0.4,1],0.1,0.02),[0,.8,.8]) for ii ∈ collect(range(-1,1,step=0.1))]
    p=rgbColor(Polygon([0,0,0.5],[1,0,0],[1,1,-0.5],[0,1,0]),[0,1,1])
    q=Torus([1/2,0,0],[0,1/2,0],[0,0,1.0],0.03)
    q=Blocks³([0,1.,0],[0,1.,1.],[0,0,1.0],0.03)
    obj=csgUnion(o,x,y,z,b,d,e,g,h,i...,j...,q)
    render(obj,name="test_axis",index=iii)
end

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:          102888   Smpls/Pxl: 0.39
Rays:             368888   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                              47490           23260     48.98
Cone/Cylinder                   117301           33054     28.18
CSG Intersection                  5894              81      1.37
Disc                            112014           12549     11.20
Sphere                            3551            2065     58.15
Torus                           528519          219609     41.55
Torus Bound

Bounding Box                   8345680         3041216     36.44
----------------------------------------------------------------------------
Roots tested:                291387   eliminated:               104374
Shadow Ray Tests:            166940   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.055 seconds)
              using 16 thread(s) with 0.496 CPU-seconds total
POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-

----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           55
Infinite Objects:          0
Light Sources:             1
Total:                    56
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.002 seconds)
              using 1 thread(s) with 0.001 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================
Rendered 250000 of 250000 pixels (100%)
------

Parser Options
  Input file: test_axis_000005.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....500 by 500 (rows 1 to 500, columns 1 to 500).
  Output file..........test_axis_000005.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           55
Infinite Objects:          0
Light Sources:        

POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:           98100   Smpls/Pxl: 0.37
Rays:             364100   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                              61166           36936     60.39


Cone/Cylinder                    92615           23434     25.30
CSG Intersection                  5894              81      1.37
Disc                            108270           12149     11.22
Sphere                            3360            2065     61.46
Torus                           581856          212009     36.44
Torus Bound                     581856          285706     49.10
Bounding Box                   8045326         2961651     36.81
----------------------------------------------------------------------------
Roots tested:                285706   eliminated:                98551
Shadow Ray Tests:            165628   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.053 seconds)
              using 16 thread

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray new_123456.pov`, ProcessExited(0))

In [35]:
q

csgIntersection(Object[AffineTransform(Box([0.737107, 0.737107, 0.03], [0.737107, 0.0, -0.03]), [0.0 -0.0 1.0; 0.707107 0.707107 0.0; -0.707107 0.707107 0.0], [0.0, 0.5, 0.5]), AffineTransform(Box([0.737107, 0.737107, 0.03], [0.737107, 0.0, -0.03]), [-0.0 -0.0 1.0; 0.707107 0.707107 0.0; -0.707107 0.707107 0.0], [0.0, 0.5, 0.5])])